In [ ]:
from typing import Any, Dict

import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel


# Assuming your model class is defined like this:
# You'll need to import your actual model class
class CreditRiskModel:
    def __init__(self, model_path: str):
        # Initialize your model
        # This is a placeholder - replace with your actual implementation
        self.model_path = model_path
        # Here you would load your model, scaler, etc.

    def predict(self, client_data: Dict[str, Any]) -> int:
        # Your prediction logic
        # This is a placeholder - replace with your actual implementation
        return 0  # or 1


# Create a Pydantic model for request validation
class ClientData(BaseModel):
    # Define the expected client data fields
    # Example fields - replace with your actual required fields
    age: int
    income: float
    credit_score: int
    loan_amount: float
    loan_term: int
    # Add more fields as needed based on your model's requirements


# Create a Pydantic model for the response
class PredictionResponse(BaseModel):
    will_repay: int  # 0 or 1
    request_id: str  # To track requests


# Initialize the FastAPI application
app = FastAPI(
    title="Credit Risk Prediction API",
    description="API for predicting if a client will repay a credit",
    version="0.1.0",
)

# Initialize your model
# Update the path to where your model is stored
# model = CreditRiskModel(model_path="path/to/your/model")

model = "path/to/your/model"


@app.post("/predict", response_model=PredictionResponse)
async def predict_credit_risk(client_data: ClientData):
    try:
        # Convert Pydantic model to dictionary
        client_dict = client_data.dict()

        # Get prediction from your model
        prediction_proba = model.predict_proba(client_dict)

        prediction = model.predict(client_dict)

        # Create a unique request ID (you may want to use UUID or another method)
        import time

        request_id = f"req_{int(time.time())}"

        # Return the prediction
        return PredictionResponse(
            proba=prediction_proba, will_repay=prediction, request_id=request_id
        )
    except Exception as e:
        # Handle exceptions
        raise HTTPException(status_code=500, detail=f"Prediction error: {str(e)}")


# Health check endpoint
@app.get("/health")
async def health_check():
    return {"status": "healthy"}


# Root endpoint with API info
@app.get("/")
async def root():
    return {
        "api": "Credit Risk Prediction API",
        "version": "0.1.0",
        "endpoints": {
            "/predict": "POST - Get credit risk prediction",
            "/health": "GET - API health check",
        },
    }


# For running the application locally
if __name__ == "__main__":
    uvicorn.run("app:app", host="0.0.0.0", port=8000, reload=True)

---

In [ ]:
import numpy as np
from fastapi import FastAPI
from pydantic import BaseModel

from projet7.class_model import Model

app = FastAPI()

# Assume the model is already trained and loaded
model = Model.get_model_info()  # Adjust this line to load your pre-trained model


class PredictionRequest(BaseModel):
    X_test: list


class PredictionResponse(BaseModel):
    probabilities: list
    binary_predictions: list


@app.post("/predict", response_model=PredictionResponse)
def predict(request: PredictionRequest):
    try:
        # Convert the input data to a numpy array
        X_test = np.array(request.X_test)

        # Get prediction probabilities
        y_pred_proba = model.best_model.predict_proba(X_test)[:, 1]

        # Apply the custom threshold to get binary predictions
        y_pred_binary = (y_pred_proba >= model.best_threshold).astype(int)

        return PredictionResponse(
            probabilities=y_pred_proba.tolist(),
            binary_predictions=y_pred_binary.tolist(),
        )
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))


@app.get("/model_info")
def get_model_info():
    try:
        # Return the model information as a JSON response
        model_info = model.get_model_info()
        return model_info
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))


if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)


---

---

In [2]:
from typing import Any, Dict

import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

from projet7.class_model import Model


# Create a Pydantic model for request validation
# we hav selected the 15 most important variables
class ClientData(BaseModel):
    ORGANIZATION_TYPE: str
    DAYS_EMPLOYED: int
    OCCUPATION_TYPE: str
    NAME_FAMILY_STATUS: str
    NAME_EDUCATION_TYPE: str
    CODE_GENDER: str
    WEEKDAY_APPR_PROCESS_START: str
    FLAG_OWN_CAR: str
    NAME_CONTRACT_TYPE: str
    NAME_INCOME_TYPE: str
    NAME_HOUSING_TYPE: str
    REGION_RATING_CLIENT: int
    WALLSMATERIAL_MODE: str
    NAME_TYPE_SUITE: str
    FLAG_OWN_REALTY: str


# Create a Pydantic model for the response
class PredictionResponse(BaseModel):
    will_repay: int  # 0 or 1
    request_id: str  # To track requests


# Initialize the FastAPI application
app = FastAPI(
    title="Credit Risk Prediction API",
    description="API for predicting if a client will repay a credit",
    version="0.1.0",
)

# Initialize your model
# Update the path to where your model is stored
# model = CreditRiskModel(model_path="path/to/your/model")

model = "../../data/processed/lgbm_model_15.pkl"


@app.post("/predict", response_model=PredictionResponse)
async def predict_credit_risk(client_data: ClientData):
    try:
        # Convert Pydantic model to dictionary
        client_dict = client_data.dict()

        # Get prediction from your model
        prediction_proba = model.predict_proba(client_dict)

        prediction = model.predict(client_dict)

        # Create a unique request ID (you may want to use UUID or another method)
        import time

        request_id = f"req_{int(time.time())}"

        # Return the prediction
        return PredictionResponse(
            proba=prediction_proba, will_repay=prediction, request_id=request_id
        )
    except Exception as e:
        # Handle exceptions
        raise HTTPException(status_code=500, detail=f"Prediction error: {str(e)}")


# Health check endpoint
@app.get("/health")
async def health_check():
    return {"status": "healthy"}


# Root endpoint with API info
@app.get("/")
async def root():
    return {
        "api": "Credit Risk Prediction API",
        "version": "0.1.0",
        "endpoints": {
            "/predict": "POST - Get credit risk prediction",
            "/health": "GET - API health check",
        },
    }


# For running the application locally
if __name__ == "__main__":
    uvicorn.run("app:app", host="0.0.0.0", port=8000, reload=True)

INFO:     Will watch for changes in these directories: ['/Users/calh/Desktop/OC/Projet7/code/notebooks']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [6637] using WatchFiles
ERROR:    Error loading ASGI app. Could not import module "app".
INFO:     Stopping reloader process [6637]


---

In [56]:
import requests

url = "http://127.0.0.1:8000/predict/"


data = {
    "inputs": [
        {
            "ORGANIZATION_TYPE": "Kindergarten",
            "DAYS_EMPLOYED": -2329,
            "OCCUPATION_TYPE": "Drivers",
            "NAME_FAMILY_STATUS": "Married",
            "NAME_EDUCATION_TYPE": "Higher education",
            "CODE_GENDER": "F",
            "WEEKDAY_APPR_PROCESS_START": "TUESDAY",
            "FLAG_OWN_CAR": "N",
            "NAME_CONTRACT_TYPE": "Cash loans",
            "NAME_INCOME_TYPE": "Working",
            "NAME_HOUSING_TYPE": "House / apartment",
            "REGION_RATING_CLIENT": 2,
            "WALLSMATERIAL_MODE": " Stone, brick",
            "NAME_TYPE_SUITE": "Unaccompanied",
            "FLAG_OWN_REALTY": "Y",
        }
    ]
}

response = requests.post(url, json=data)
print(response.json())


{'probabilities': [0.05388329670190144], 'binary_predictions': [0]}


In [ ]:
def cleaning(features, print_shape=True):
    """Fonction cleaning finale using sklearn Pipeline"""
    categorical_columns = features.select_dtypes(
        include=["object", "category"]
    ).columns.tolist()
    numerical_columns = features.select_dtypes(
        include=["int64", "float64"]
    ).columns.tolist()

    preprocessor = joblib.load("../../data/processed/preprocessor_top15.joblib")

    # Fit the preprocessor to the training data and transform both datasets
    features_transformed = preprocessor.transform(features)

    # Get feature names after one-hot encoding
    # First, get numerical feature names directly
    feature_names = numerical_columns.copy()

    # Then add categorical feature names after one-hot encoding
    if categorical_columns:
        ohe = preprocessor.named_transformers_["categorical"].named_steps["encoder"]
        categorical_feature_names = ohe.get_feature_names_out(categorical_columns)
        feature_names.extend(categorical_feature_names)

    return features_transformed


In [ ]:
import joblib
import pandas as pd
from pydantic import BaseModel

data = {
    "inputs": [
        {
            "ORGANIZATION_TYPE": "Kindergarten",
            "DAYS_EMPLOYED": -2329,
            "OCCUPATION_TYPE": "Drivers",
            "NAME_FAMILY_STATUS": "Married",
            "NAME_EDUCATION_TYPE": "Higher education",
            "CODE_GENDER": "F",
            "WEEKDAY_APPR_PROCESS_START": "TUESDAY",
            "FLAG_OWN_CAR": "N",
            "NAME_CONTRACT_TYPE": "Cash loans",
            "NAME_INCOME_TYPE": "Working",
            "NAME_HOUSING_TYPE": "House / apartment",
            "REGION_RATING_CLIENT": 2,
            "WALLSMATERIAL_MODE": " Stone, brick",
            "NAME_TYPE_SUITE": "Unaccompanied",
            "FLAG_OWN_REALTY": "Y",
        }
    ]
}

model_info = joblib.load("../../data/processed/lgbm_model_15_info.joblib")

input_data = pd.DataFrame(data["inputs"])

# Apply your preprocessing function
X_test_processed = cleaning(input_data)

y_pred_proba = model_info["best_model"].predict_proba(X_test_processed)[:, 1]


/Users/calh/Desktop/OC/Projet7/.pixi/envs/default/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
